In [3]:
import nilearn
import sklearn
from nilearn import plotting
from nilearn import datasets
from nilearn import image
from nilearn.image import mean_img
from nilearn.image import index_img
from nilearn.glm.first_level import make_first_level_design_matrix
from nilearn.plotting import plot_design_matrix

from sklearn.model_selection import KFold
from nilearn.decoding import Decoder 

from sklearn import svm
from sklearn.svm import SVC
from nilearn.input_data import NiftiMasker
from sklearn.model_selection import cross_val_score

import numpy as np
import pandas as pd
import os


/Users/home/.local/lib/python3.8/site-packages/nilearn/datasets/__init__.py:87: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  warn("Fetchers from the nilearn.datasets module will be "


In [4]:
# Set variables
ROOT_DATA='/Users/home/Documents/BIDS/'
#ROOT_DATA='/home/brunomiguel/Documents/data/BIDS/'
SUB='sub-0001'
SES='ses-001'
TASK='innerspeech'
RUN='run-02'


# data folder
data_path=os.path.join(ROOT_DATA, SUB, SES)
print('The data is in this folder - ' + data_path)

# project folder
ROOT_PROJECT="/Volumes/LANGLYBACKU/MVPA_Speech_project"


The data is in this folder - /Users/home/Documents/BIDS/sub-0001/ses-001


In [5]:

# Load mask
mask_fn=os.path.join(SUB + '_' + SES + '_task-' + TASK + '_'+ RUN +'_mask.nii.gz')
brain_mask= image.load_img(mask_fn)

# Load data
bs_fn=os.path.join(SUB + '_' + SES + '_task-' + TASK + '_'+ RUN +'_bold_bs.nii.gz')
bs_maps= image.load_img(bs_fn)

# Load labels
labels_fn=os.path.join(SUB + '_' + SES + '_task-' + TASK + '_'+ RUN +'_labels.csv')
col_list = ["0"]
labels=pd.read_csv(labels_fn, usecols=col_list)


In [6]:
bs_maps.shape


(91, 109, 91, 252)

In [7]:
conditions=labels['0']

## Decoder

In [8]:
svc = SVC()
cv=

In [9]:
masker_f = NiftiMasker(mask_img=brain_mask, standardize=True)
fmri_masked = masker_f.fit_transform(bs_maps)
print(fmri_masked.shape)

(252, 7797)


In [10]:
t=np.hstack((fmri_masked,fmri_masked))
print(t.shape)

(252, 15594)


In [ ]:
masker_all = NiftiMasker(standardize=True)
fmri_all = masker_all.fit_transform(bs_maps)
print(fmri_all.shape)

In [ ]:
lin_clf = svm.LinearSVC()
lin_clf.fit(fmri_all, conditions)


In [ ]:
lin_clf.coef_.shape

In [ ]:
cv_scores = cross_val_score(lin_clf, fmri_masked, conditions, cv=cv, scoring="balanced_accuracy")
print("Model accuracy: {:.3f}".format(cv_scores.mean()))

In [ ]:

cv_scores = cross_val_score(svc, fmri_masked, conditions, cv=cv, scoring="balanced_accuracy")
print("Model accuracy: {:.3f}".format(cv_scores.mean()))

cv_scores_all = cross_val_score(svc, fmri_all, conditions, cv=cv, scoring="balanced_accuracy")
print("Model accuracy: {:.3f}".format(cv_scores.mean()))

In [ ]:
svc_coef = lin_clf.coef_

weight_img = masker_all.inverse_transform(svc_coef[0,:])


plot_stat_map(weight_img, title='SVC weights',threshold=.005)

#mean_svc_coef=mean_img(svc_coef)
#plot_stat_map(mean_svc_coef, title='SVC weights')

In [ ]:
lin_clf.predict(fmri_all)

In [ ]:
from sklearn.dummy import DummyClassifier
null_cv_scores = cross_val_score(
    DummyClassifier(), fmri_all, conditions, cv=cv)

print("Dummy accuracy: {:.3f}".format(null_cv_scores.mean()))

## Decoding without mask

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectPercentile, f_classif
from sklearn.model_selection import cross_validate
from sklearn.svm import LinearSVC
feature_selection = SelectPercentile(f_classif, percentile=2)
anova_svc = Pipeline([('anova', feature_selection), ('svc', LinearSVC())])
# We can use our ``anova_svc`` object exactly as we were using our ``svc``
# object previously.
# As we want to investigate our model, we use sklearn `cross_validate` function
# with `return_estimator = True` instead of cross_val_score, to save the estimator

fitted_pipeline = cross_validate(anova_svc, fmri_all, labels,
                                 cv=cv, return_estimator=True)

print(
    "ANOVA+SVC test score: {:.3f}".format(fitted_pipeline["test_score"].mean()))


In [ ]:
# retrieve the pipeline fitted on the first cross-validation fold and its SVC
# coefficients
first_pipeline = fitted_pipeline["estimator"][0]
svc_coef = first_pipeline.named_steps['svc'].coef_
print("After feature selection, the SVC is trained only on {} features".format(
    svc_coef.shape[1]))

# We invert the feature selection step to put these coefs in the right 2D place
full_coef = first_pipeline.named_steps['anova'].inverse_transform(svc_coef)

print("After inverting feature selection, we have {} features back".format(
    full_coef.shape[1]))

# We apply the inverse of masking on these to make a 4D image that we can plot
from nilearn.plotting import plot_stat_map
weight_img = masker.inverse_transform(full_coef)
mean_weight_img=mean_img(weight_img)
plot_stat_map(mean_weight_img, title='Anova+SVC weights')